In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from sklearn.metrics import mean_squared_error


df= pd.read_json("Automotive_5.json",lines = True)
df = df[['asin','overall','reviewerID']].copy()

In [4]:
df_asin = df.groupby('asin').count()
df_asin = df_asin.index.values
df_asin

array(['B00002243X', 'B00002243Z', 'B00008BKX5', ..., 'B00JGM7Z5Q',
       'B00JGP8I1S', 'B00KIAQ8VW'], dtype=object)

In [5]:
df_reviewerID = df.groupby('reviewerID').count()
df_reviewerID = df_reviewerID.index.values
df_reviewerID

array(['A00473363TJ8YSZ3YAGG9', 'A0473259F6GQNBD88IYN',
       'A09567722SXKYZTV7OFCJ', ..., 'AZTAK8PFB0NLV', 'AZV2U6GU5QA6C',
       'AZYBZEVISD0YX'], dtype=object)

In [6]:
df_dict = {}
for line in df.values:
    if line[0] in df_dict:
        df_dict[line[0]].append(line[2])
    else:
        df_dict[line[0]] = [line[2]]

In [8]:
print(len(df_asin))
print(len(df_reviewerID))
print(len(df_dict))

1835
2928
1835


In [9]:
predict_for_this = []
count = 0
for i in range(len(df_reviewerID)):
    for j in range(len(df_asin)):
        if not df_reviewerID[i] in df_dict[df_asin[j]]:
            predict_for_this.append((df_reviewerID[i],df_asin[j]))
            
len(predict_for_this)

5352407

In [14]:
df_new = df.pivot(index = "asin" , columns = "reviewerID" , values = "overall")
# df_original = df_new.copy()
df_new = df_new.sort_index(axis=1)

In [15]:
X = df_new.values
Y = df_new.sum(axis = 1)
Z =df_new.count(axis =1).astype('float64')
Y = Y.div(Z,axis=0)
df_new = df_new.sub(Y,axis=0).fillna(0)

In [16]:
nbrs_train = NearestNeighbors(n_neighbors= 5, algorithm='auto',metric ='cosine').fit(df_new.values)

In [17]:
distances, indices = nbrs_train.kneighbors(df_new,return_distance=True)

In [20]:
item_avg = df.groupby(['asin']).mean()

In [22]:
user_avg = df.groupby(['reviewerID']).mean()

In [23]:
mean_avg = df["overall"].mean()

In [26]:
item_map = df_new.index.values
item_map

array(['B00002243X', 'B00002243Z', 'B00008BKX5', ..., 'B00JGM7Z5Q',
       'B00JGP8I1S', 'B00KIAQ8VW'], dtype=object)

In [29]:
predict_for_this
final_prediction = {}

for i in range(len(predict_for_this)):
#__________________________________________________________________________________________________________
# for i in range(len(predict_for_this)): This loop needs to be run instead of above one. 5352407 records: takes time
#__________________________________________________________________________________________________________
    
    item_asin = predict_for_this[i][1]
    item_userID = predict_for_this[i][0]
#     print(item_asin,item_userID)
    
    predicted_rating = 0
    if item_asin in item_avg.index:
        bi = item_avg.loc[item_asin]['overall'] - mean_avg
    else:
        bi = 0
    if item_userID in user_avg.index:
        bx = user_avg.loc[item_userID]['overall'] - mean_avg
    else:
        bx = 0
            
    baseline_estimate = mean_avg + bi + bx
    predicted_rating = baseline_estimate
#     print(baseline_estimate)
    
    numerator = 0 
    denomenator = 0
    
    if bi:
        
        item  = np.where(item_map == item_asin)[0][0]
        item_nbrs = indices[item:item+1]
        dist_nbrs = distances[item:item+1]
        cnt = 0;
        k = 0
        
        for j in item_nbrs:
            temp = df.loc[(df['asin'] == item_map[j[0]]) &  (df['reviewerID'] == item_userID)]
            if(temp.size != 0):
                dist = dist_nbrs[cnt:cnt+1]
                numerator += ((1 - dist[0][k])*(temp['overall'].values[0] - (item_avg.loc[temp['asin']]['overall'] - mean_avg) ))

                denomenator += (1 - dist[0][k])
            cnt = cnt + 1
            k+= 1
        if denomenator:
            predicted_rating += numerator/denomenator 

    if item_userID in final_prediction:
        final_prediction[item_userID].append((item_asin,predicted_rating))
    else:
        final_prediction[(item_userID)] = [(item_asin,predicted_rating)]

#___________________________________________________________
#        Sort final_prediction on basis of rating
#___________________________________________________________

    
print(final_prediction)

KeyboardInterrupt: 